In [ ]:
import pandas as pd
import joblib
import os
import gc

In [ ]:
def predict(data, models):

    model_list = []
    for path in ['models_DART_all_56corr_pcaslope_lag_SEED42/']:
        for fname in os.listdir(path):
            for model_name in models:
                if model_name in fname:
                    model_list.append(path + fname)

    pred_list = []
    for model_path in model_list:
        print(model_path)
        print('{:.02f} MB'.format(os.path.getsize(model_path)/1000000))
        model = joblib.load(model_path)
        pred_list.append(model.predict(data))
    
    return pred_list, model_list

In [ ]:
test = pd.read_parquet('Data/test_all_slopes_corr_pcaslope_lag.parquet')
corr_col = test.columns[test.columns.str.startswith('corr_')].to_list()


top_corr = [
    "corr_D_39-B_26",
    "corr_D_48-B_4",
    "corr_P_2-D_44",
    "corr_D_47-B_4",
    "corr_D_47-D_39",
    "corr_P_2-B_4",
    "corr_D_39-B_10",
    "corr_D_44-B_4",
    "corr_D_39-B_2",
    "corr_D_46-B_4",
    "corr_D_48-B_3",
    "corr_D_48-B_9",
    "corr_S_5-S_24",
    "corr_S_7-S_3",
    "corr_D_43-D_144",
    "corr_D_48-D_39",
    "corr_P_3-D_46",
    "corr_S_5-D_43",
    "corr_R_1-B_4",
    "corr_P_3-D_47",
    "corr_D_39-B_3",
    "corr_R_6-D_39",
    "corr_S_27-B_2",
    "corr_S_23-D_43",
    "corr_R_6-D_69",
    "corr_P_2-D_48",
    "corr_S_25-B_4",
    "corr_D_43-B_4",
    "corr_R_27-D_69",
    "corr_S_7-S_27",
    "corr_D_39-B_11",
    "corr_S_3-D_39",
    "corr_S_12-B_4",
    "corr_D_39-B_15",
    "corr_R_27-B_26",
    "corr_S_23-D_39",
    "corr_R_27-R_1",
    "corr_R_1-D_39",
    "corr_S_19-D_39",
    "corr_S_27-B_3",
    "corr_S_16-D_39",
    "corr_R_27-B_5",
    "corr_S_3-D_62",
    "corr_D_71-D_62",
    "corr_R_27-D_39",
    "corr_D_48-D_43",
    "corr_D_61-B_36",
    "corr_S_25-D_39",
    "corr_R_6-D_43",
    "corr_S_27-R_27",
    "corr_S_27-S_12",
    "corr_S_27-D_39",
    "corr_D_46-B_3",
    "corr_D_62-D_47",
    "corr_B_4-B_3",
    "corr_R_1-D_48",
]

corr_to_remove = set(corr_col).difference(set(top_corr))
test.drop(corr_to_remove, axis=1, inplace=True)

models = [
    'HT',
]

pred_list, model_list = predict(test, models)

In [ ]:
model_names = [model.split('/')[-1][:10] for model in model_list]
pred_df = pd.DataFrame(pred_list).T
pred_df.columns = model_names
pred_df.index = test.index

del test
_ = gc.collect()

In [ ]:
sorted_columns = sorted(pred_df.columns)
pred_df = pred_df[sorted_columns]

In [ ]:
pred_df = pd.read_csv('models_DART_all_56corr_pcaslope_lag_SEED42/p_M6_folds.csv', index_col='customer_ID')

In [ ]:
pred_df.to_csv('models_DART_all_56corr_pcaslope_lag_SEED42/p_M6_folds.csv')

In [ ]:
HT = ['HT2_fold_0', 'HT1_fold_1', 'HT2_fold_2', 'HT1_fold_3', 'HT2_fold_4']
pred_df[HT]

In [ ]:
pred_df[HT].mean(axis=1).to_csv('models_DART_all_56corr_pcaslope_lag_SEED42/p_HT2.csv', header=['prediction'])

In [ ]:
p1 = pd.read_csv('models_DART_all_56corr_pcaslope_lag_SEED42/p_HT2.csv', index_col='customer_ID')
p2 = pd.read_csv('Output/test_lgbm_baseline_5fold_seed_blend.csv', index_col='customer_ID')

In [ ]:
ensemble = p1['prediction'] * 0.50 + p2['prediction'] * 0.50
ensemble.to_csv('Output/ensemble_HT2_p0_0.5.csv', header=['prediction'])